## SECTION 1

In [1]:
# ============================================================================
# SECTION 1: ENVIRONMENT SETUP AND INSTALLATIONS  (PINNED VERSIONS)
# ============================================================================

import sys, subprocess, importlib

def pip_install(pkgs):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", *pkgs])

print("Installing pinned libraries …")
pip_install([
    "transformers==4.44.2",
    "accelerate==0.34.2",
    "datasets==2.21.0",
    "scikit-learn==1.5.2",
    "pandas==2.2.3",
    "numpy==1.26.4",
    "matplotlib==3.9.2",
    # Torch: if you already have CUDA-enabled torch, you can skip this next line.
    # If Colab/VM has no torch, uncomment one of the two lines below:
    # "torch==2.2.2", "torchaudio==2.2.2", "torchvision==0.17.2"
])

# Optional tools
# pip_install(["wandb"])

# --- Print versions to confirm ---
import torch, transformers, datasets, sklearn, pandas as pd, numpy as np, matplotlib

print("\n=== VERSION CHECK ===")
print("torch          :", getattr(torch, "__version__", "n/a"))
print("transformers   :", transformers.__version__)
print("accelerate     :", importlib.import_module("accelerate").__version__)
print("datasets       :", datasets.__version__)
print("scikit-learn   :", sklearn.__version__)
print("pandas         :", pd.__version__)
print("numpy          :", np.__version__)
print("matplotlib     :", matplotlib.__version__)

# Sanity: ensure transformers is new enough for TrainingArguments(evaluation_strategy=…)
from packaging import version
assert version.parse(transformers.__version__) >= version.parse("4.26.0"), (
    "Transformers too old for `evaluation_strategy`. Please re-run this cell to upgrade."
)

# GPU info (optional)
print("\nCUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA Device Count:", torch.cuda.device_count())
    print("Current CUDA Device:", torch.cuda.get_device_name(0))


Installing pinned libraries …


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]


=== VERSION CHECK ===
torch          : 2.8.0+cu126
transformers   : 4.44.2
accelerate     : 0.34.2
datasets       : 2.21.0
scikit-learn   : 1.5.2
pandas         : 2.2.3
numpy          : 1.26.4
matplotlib     : 3.9.2

CUDA Available: True
CUDA Device Count: 1
Current CUDA Device: Tesla T4


### SECTION 1.5


In [2]:
# ============================================================================
# SECTION 1.5: VERSION CHECK + TRAININGARGUMENTS COMPATIBILITY SHIM
# ============================================================================

import inspect, importlib, sys
import transformers as _tf

print("Transformers version loaded in memory:", _tf.__version__)

def _supported_kwargs_of_training_args():
    # Build the set of supported __init__ kwargs for the loaded TrainingArguments
    try:
        from transformers import TrainingArguments
        sig = inspect.signature(TrainingArguments.__init__)
        return set(sig.parameters.keys())
    except Exception as e:
        print("[Compat] Could not inspect TrainingArguments:", e)
        return set()

_SUPPORTED_TA_KEYS = _supported_kwargs_of_training_args()
print("Sample of supported TrainingArguments kwargs:", sorted(list(_SUPPORTED_TA_KEYS))[:12], "...")

def make_training_args_compat(**kwargs):
    """
    Create TrainingArguments while dropping any kwargs unsupported by the loaded transformers version.
    Prints what was ignored so you know if your runtime is old.
    """
    from transformers import TrainingArguments
    filtered = {k: v for k, v in kwargs.items() if k in _SUPPORTED_TA_KEYS}
    ignored = [k for k in kwargs.keys() if k not in _SUPPORTED_TA_KEYS]
    if ignored:
        print("[Compat] Ignored unsupported TrainingArguments keys:", ignored)
    return TrainingArguments(**filtered)

def get_early_stopping_callbacks(patience: int):
    """Return EarlyStoppingCallback if available; otherwise return []."""
    try:
        from transformers import EarlyStoppingCallback
        return [EarlyStoppingCallback(early_stopping_patience=patience)]
    except Exception as e:
        print("[Compat] EarlyStoppingCallback unavailable:", e)
        return []


Transformers version loaded in memory: 4.44.2
Sample of supported TrainingArguments kwargs: ['accelerator_config', 'adafactor', 'adam_beta1', 'adam_beta2', 'adam_epsilon', 'auto_find_batch_size', 'batch_eval_metrics', 'bf16', 'bf16_full_eval', 'data_seed', 'dataloader_drop_last', 'dataloader_num_workers'] ...


## SECTION 2

In [3]:

# ============================================================================
# SECTION 2: IMPORTS AND BASIC SETUP
# ============================================================================

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
import random
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)


In [4]:
import os, random, json, math
from dataclasses import dataclass
from typing import Dict, Tuple, Optional, List

import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt

from transformers import (
    AutoTokenizer, AutoModel, TrainingArguments, Trainer,
    DataCollatorWithPadding, EarlyStoppingCallback
)

def seed_all(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_all(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

## SECTION 3

In [5]:
# SECTION 3

data_path = '/content/adjudications_2025-10-21.csv'
CSV_PATH = '/content/adjudications_2025-10-21.csv'

# Your exact column names
TITLE_COL = "Title"
TEXT_COL  = "Comment"
SENT_COL  = "Final Sentiment"
POL_COL   = "Final Polarization"

MODEL_CONFIGS = {
    "mbert":       {"name": "bert-base-multilingual-cased", "desc": "mBERT (104 langs)"},
    "xlm_roberta": {"name": "xlm-roberta-base",             "desc": "XLM-R base"},
    "rembert":     {"name": "google/rembert",               "desc": "RemBERT"},
}
MODELS_TO_RUN = ["mbert", "xlm_roberta"]

# Training hyperparams
MAX_LENGTH = 224
EPOCHS = 4
BATCH_SIZE = 16
LR = 2e-5
WEIGHT_DECAY = 0.01
WARMUP_RATIO = 0.06
EARLY_STOP_PATIENCE = 2

# --- Per-task loss config (these worked well) ---
USE_FOCAL_SENTIMENT = False
USE_FOCAL_POLARITY  = True
FOCAL_GAMMA_POLARITY = 1.2
LABEL_SMOOTH_SENTIMENT = 0.05

# Task loss weighting (keep polarity gentle)
TASK_LOSS_WEIGHTS = {"sentiment": 1.0, "polarization": 1.05}

# Per-class multipliers (on top of balanced weights from train split)
CLASS_WEIGHT_MULT = {
    "sentiment": {
        "negative": 1.40,
        "neutral":  1.00,
        "positive": 1.10
    },
    "polarization": {
        "non_polarized": 1.00,
        "objective":     2.00,
        "partisan":      1.00
    }
}

# Oversampling (blended joint)
USE_OVERSAMPLING = True
USE_JOINT_OVERSAMPLING = True
JOINT_ALPHA = 0.40
JOINT_OVERSAMPLING_MAX_MULT = 3.0

# --- NEW: small MLP head before classifiers ---
HEAD_HIDDEN = 256
HEAD_DROPOUT = 0.15

# Output dir
OUT_DIR = "./runs_multitask"
os.makedirs(OUT_DIR, exist_ok=True)



## SECTION 4

In [6]:
# ===== Section 4 — Load & Prepare Data (updated for multipliers) =====
df = pd.read_csv(CSV_PATH)
df.columns = df.columns.str.strip()

required = [TITLE_COL, TEXT_COL, SENT_COL, POL_COL]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Missing expected columns: {missing}. Found: {list(df.columns)}")

df = df.dropna(subset=[TITLE_COL, TEXT_COL, SENT_COL, POL_COL]).reset_index(drop=True)

# Encode labels
from sklearn.preprocessing import LabelEncoder
sent_le = LabelEncoder().fit(df[SENT_COL])
pol_le  = LabelEncoder().fit(df[POL_COL])

df["sent_y"] = sent_le.transform(df[SENT_COL])
df["pol_y"]  = pol_le.transform(df[POL_COL])

num_sent_classes = len(sent_le.classes_)
num_pol_classes  = len(pol_le.classes_)

print("Sentiment classes:", dict(enumerate(sent_le.classes_)))
print("Polarization classes:", dict(enumerate(pol_le.classes_)))

# Splits (stratify by sentiment)
from sklearn.model_selection import train_test_split
X = df[[TITLE_COL, TEXT_COL]].copy()
y_sent = df["sent_y"].values
y_pol  = df["pol_y"].values

X_train, X_tmp, ysent_train, ysent_tmp, ypol_train, ypol_tmp = train_test_split(
    X, y_sent, y_pol, test_size=0.3, random_state=42, stratify=y_sent
)
X_val, X_test, ysent_val, ysent_test, ypol_val, ypol_test = train_test_split(
    X_tmp, ysent_tmp, ypol_tmp, test_size=0.5, random_state=42, stratify=ysent_tmp
)

print("Train size:", len(X_train), "Val size:", len(X_val), "Test size:", len(X_test))

# Balanced class weights from TRAIN only
from sklearn.utils.class_weight import compute_class_weight
import numpy as np, json, os

def safe_class_weights(y, n_classes):
    classes = np.arange(n_classes)
    counts = np.bincount(y, minlength=n_classes)
    if np.any(counts == 0):
        return np.ones(n_classes, dtype=np.float32)
    return compute_class_weight("balanced", classes=classes, y=y).astype(np.float32)

sent_weights_np = safe_class_weights(ysent_train, num_sent_classes)
pol_weights_np  = safe_class_weights(ypol_train,  num_pol_classes)

# Apply user multipliers by class name
sent_name_to_idx = {name: i for i, name in enumerate(sent_le.classes_)}
pol_name_to_idx  = {name: i for i, name in enumerate(pol_le.classes_)}

for cname, mult in CLASS_WEIGHT_MULT["sentiment"].items():
    if cname in sent_name_to_idx:
        sent_weights_np[sent_name_to_idx[cname]] *= float(mult)

for cname, mult in CLASS_WEIGHT_MULT["polarization"].items():
    if cname in pol_name_to_idx:
        pol_weights_np[pol_name_to_idx[cname]] *= float(mult)

print("Final sentiment class weights:", {sent_le.classes_[i]: float(w) for i, w in enumerate(sent_weights_np)})
print("Final polarization class weights:", {pol_le.classes_[i]: float(w) for i, w in enumerate(pol_weights_np)})

# Save label maps
with open(os.path.join(OUT_DIR, "label_map_sentiment.json"), "w") as f:
    json.dump({int(k): v for k, v in dict(enumerate(sent_le.classes_)).items()}, f, indent=2)
with open(os.path.join(OUT_DIR, "label_map_polarization.json"), "w") as f:
    json.dump({int(k): v for k, v in dict(enumerate(pol_le.classes_)).items()}, f, indent=2)


Sentiment classes: {0: 'negative', 1: 'neutral', 2: 'positive'}
Polarization classes: {0: 'non_polarized', 1: 'objective', 2: 'partisan'}
Train size: 5691 Val size: 1220 Test size: 1220
Final sentiment class weights: {'negative': 0.8092016577720642, 'neutral': 1.1841448545455933, 'positive': 2.585749626159668}
Final polarization class weights: {'non_polarized': 1.1061224937438965, 'objective': 11.126099586486816, 'partisan': 0.5218706727027893}


## SECTION 5

In [7]:
# ===== Section 5 — Dataset & Collator (proper text-pair encoding) =====
from torch.utils.data import Dataset

class TaglishDataset(Dataset):
    def __init__(self, titles, texts, y_sent, y_pol, tokenizer, max_length=224):
        self.titles = list(titles)
        self.texts  = list(texts)
        self.y_sent = np.array(y_sent)
        self.y_pol  = np.array(y_pol)
        self.tok = tokenizer
        self.max_length = max_length
        # mBERT has token_type_ids; XLM-R/RemBERT don't, and that's fine.
        self.use_token_type = "token_type_ids" in tokenizer.model_input_names

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Pass title as text, comment as text_pair so the tokenizer inserts the correct separators.
        # We also bias truncation to the comment since titles are short.
        enc = self.tok(
            text=str(self.titles[idx]),
            text_pair=str(self.texts[idx]),
            truncation="only_second",     # keep the title intact; trim the comment if needed
            max_length=self.max_length,
            return_token_type_ids=self.use_token_type,
        )
        item = {
            "input_ids": enc["input_ids"],
            "attention_mask": enc["attention_mask"],
            "sentiment_labels": torch.tensor(self.y_sent[idx], dtype=torch.long),
            "polarization_labels": torch.tensor(self.y_pol[idx], dtype=torch.long),
        }
        if self.use_token_type and "token_type_ids" in enc:
            item["token_type_ids"] = enc["token_type_ids"]
        return item


## SECTION 6

In [8]:
# ===== Section 6 — Multi-Task Model (shared encoder + 2-layer head) =====
import torch.nn as nn
from transformers import AutoModel

class MultiTaskModel(nn.Module):
    def __init__(self, base_model_name: str, num_sent: int, num_pol: int, dropout: float = 0.1):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(base_model_name)
        hidden = self.encoder.config.hidden_size

        # Head trunk (projection): [hidden] -> [HEAD_HIDDEN]
        self.trunk = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden, HEAD_HIDDEN),
            nn.GELU(),
            nn.LayerNorm(HEAD_HIDDEN),
            nn.Dropout(HEAD_DROPOUT),
        )

        # Task heads on top of the shared trunk
        self.head_sent = nn.Linear(HEAD_HIDDEN, num_sent)
        self.head_pol  = nn.Linear(HEAD_HIDDEN, num_pol)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        sentiment_labels=None,
        polarization_labels=None
    ):
        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids if token_type_ids is not None else None
        )
        # Prefer pooler if available; else CLS
        pooled = getattr(outputs, "pooler_output", None)
        if pooled is None:
            pooled = outputs.last_hidden_state[:, 0]

        z = self.trunk(pooled)
        sent_logits = self.head_sent(z)
        pol_logits  = self.head_pol(z)

        return {"logits": (sent_logits, pol_logits)}


## SECTION 7

In [9]:
# SECTION 7

def compute_metrics_multi(eval_pred):
    (sent_logits, pol_logits) = eval_pred.predictions
    (y_sent, y_pol) = eval_pred.label_ids

    ps = np.argmax(sent_logits, axis=1)
    pp = np.argmax(pol_logits, axis=1)

    # Macro metrics
    sent_report = classification_report(y_sent, ps, output_dict=True, zero_division=0)
    pol_report  = classification_report(y_pol,  pp, output_dict=True, zero_division=0)

    sent_f1 = sent_report["macro avg"]["f1-score"]
    pol_f1  = pol_report["macro avg"]["f1-score"]
    macro_f1_avg = (sent_f1 + pol_f1) / 2.0

    return {
        "sent_acc": sent_report["accuracy"],
        "sent_prec": sent_report["macro avg"]["precision"],
        "sent_rec": sent_report["macro avg"]["recall"],
        "sent_f1": sent_f1,

        "pol_acc": pol_report["accuracy"],
        "pol_prec": pol_report["macro avg"]["precision"],
        "pol_rec": pol_report["macro avg"]["recall"],
        "pol_f1": pol_f1,

        "macro_f1_avg": macro_f1_avg
    }


## SECTION 8

In [10]:
# ===== Section 8 — Custom Trainer (per-task losses) =====
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer
from torch.utils.data import DataLoader

class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=2.0, reduction="mean"):
        super().__init__()
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction
    def forward(self, logits, target):
        logp = F.log_softmax(logits, dim=1)
        p = torch.exp(logp)
        focal = (1 - p) ** self.gamma
        loss = F.nll_loss(focal * logp, target, weight=self.weight, reduction="none")
        if self.reduction == "mean":
            return loss.mean()
        elif self.reduction == "sum":
            return loss.sum()
        return loss

class MultiTaskTrainer(Trainer):
    def __init__(self, *args, class_weights=None, task_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights or {}
        self.task_weights  = task_weights or {"sentiment": 1.0, "polarization": 1.0}
        self._custom_train_sampler = None

    def set_train_sampler(self, sampler):
        self._custom_train_sampler = sampler

    def get_train_dataloader(self):
        if self.train_dataset is None:
            return None
        if self._custom_train_sampler is not None:
            return DataLoader(
                self.train_dataset,
                batch_size=self.args.train_batch_size,
                sampler=self._custom_train_sampler,
                collate_fn=self.data_collator,
                drop_last=self.args.dataloader_drop_last,
                num_workers=self.args.dataloader_num_workers,
                pin_memory=self.args.dataloader_pin_memory,
            )
        return super().get_train_dataloader()

    def _sent_loss_fn(self, weight, logits, target):
        # Sentiment: CE with small label smoothing
        return nn.CrossEntropyLoss(weight=weight, label_smoothing=float(LABEL_SMOOTH_SENTIMENT))(logits, target)

    def _pol_loss_fn(self, weight, logits, target):
        # Polarity: focal loss to fight majority dominance
        return FocalLoss(weight=weight, gamma=FOCAL_GAMMA_POLARITY)(logits, target)

    def compute_loss(self, model, inputs, return_outputs=False):
        y_sent = inputs.pop("sentiment_labels")
        y_pol  = inputs.pop("polarization_labels")

        outputs = model(**inputs)
        sent_logits, pol_logits = outputs["logits"]

        # prepare weights on device
        ws = self.class_weights.get("sentiment", None)
        wp = self.class_weights.get("polarization", None)
        ws = ws.to(sent_logits.device) if ws is not None else None
        wp = wp.to(pol_logits.device)  if wp is not None else None

        # per-task losses
        loss_sent = self._sent_loss_fn(ws, sent_logits, y_sent) if not USE_FOCAL_SENTIMENT \
                    else FocalLoss(weight=ws, gamma=1.5)(sent_logits, y_sent)
        loss_pol  = self._pol_loss_fn(wp,  pol_logits,  y_pol)  if USE_FOCAL_POLARITY \
                    else nn.CrossEntropyLoss(weight=wp)(pol_logits, y_pol)

        # task-weighted sum
        w_s = float(self.task_weights.get("sentiment", 1.0))
        w_p = float(self.task_weights.get("polarization", 1.0))
        loss = w_s * loss_sent + w_p * loss_pol

        if return_outputs:
            outputs = dict(outputs)
            outputs["labels"] = (y_sent, y_pol)
            return loss, outputs
        return loss

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        y_sent = inputs.get("sentiment_labels")
        y_pol  = inputs.get("polarization_labels")

        model_inputs = {"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"]}
        if "token_type_ids" in inputs:
            model_inputs["token_type_ids"] = inputs["token_type_ids"]

        model.eval()
        with torch.no_grad():
            outputs = model(**model_inputs)
            sent_logits, pol_logits = outputs["logits"]

        loss = None
        logits = (sent_logits.detach(), pol_logits.detach())
        labels = (y_sent, y_pol)
        return (loss, logits, labels)


## SECTION 9

In [11]:
# ===== Section 9 — Train/Evaluate One Model (blended joint oversampling + compat) =====
from transformers import AutoTokenizer, DataCollatorWithPadding
import math, json, numpy as np, pandas as pd, os
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import torch
from torch.utils.data import WeightedRandomSampler
from collections import Counter

def train_eval_one_model(model_key: str,
                         X_tr: pd.DataFrame, X_v: pd.DataFrame, X_te: pd.DataFrame,
                         ysent_tr: np.ndarray, ysent_v: np.ndarray, ysent_te: np.ndarray,
                         ypol_tr: np.ndarray,  ypol_v: np.ndarray,  ypol_te: np.ndarray,
                         sent_w_np: np.ndarray, pol_w_np: np.ndarray):
    base_name = MODEL_CONFIGS[model_key]["name"]
    run_dir = os.path.join(OUT_DIR, f"{model_key}")
    os.makedirs(run_dir, exist_ok=True)

    tokenizer = AutoTokenizer.from_pretrained(base_name)
    collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

    tr_titles, tr_texts = X_tr[TITLE_COL].values, X_tr[TEXT_COL].values
    v_titles,  v_texts  = X_v[TITLE_COL].values,  X_v[TEXT_COL].values
    te_titles, te_texts = X_te[TITLE_COL].values, X_te[TEXT_COL].values

    train_ds = TaglishDataset(tr_titles, tr_texts, ysent_tr, ypol_tr, tokenizer, max_length=MAX_LENGTH)
    val_ds   = TaglishDataset(v_titles,  v_texts,  ysent_v,  ypol_v,  tokenizer, max_length=MAX_LENGTH)
    test_ds  = TaglishDataset(te_titles, te_texts, ysent_te, ypol_te, tokenizer, max_length=MAX_LENGTH)

    # Model
    model = MultiTaskModel(base_name, num_sent_classes, num_pol_classes).to(device)

    # Class weights (torch tensors)
    sent_w = torch.tensor(sent_w_np, dtype=torch.float32)
    pol_w  = torch.tensor(pol_w_np,  dtype=torch.float32)

    # --- Build TrainingArguments through the compat shim ---
    args = make_training_args_compat(
        output_dir=run_dir,
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        learning_rate=LR,
        weight_decay=WEIGHT_DECAY,
        warmup_ratio=WARMUP_RATIO,
        evaluation_strategy="epoch",     # dropped silently if unsupported
        save_strategy="epoch",           # dropped silently if unsupported
        load_best_model_at_end=True,     # dropped silently if unsupported
        metric_for_best_model="macro_f1_avg",
        greater_is_better=True,
        fp16=torch.cuda.is_available(),
        logging_dir=os.path.join(run_dir, "logs"),
        logging_steps=50,
        report_to="none",
        seed=42,
        remove_unused_columns=False,     # dropped silently if unsupported
        eval_accumulation_steps=1
    )

    callbacks = get_early_stopping_callbacks(EARLY_STOP_PATIENCE)

    trainer = MultiTaskTrainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_multi,
        callbacks=callbacks,
        class_weights={"sentiment": sent_w, "polarization": pol_w},
        task_weights=TASK_LOSS_WEIGHTS
    )

    # ----- BLENDED JOINT OVERSAMPLING -----
    if USE_OVERSAMPLING and USE_JOINT_OVERSAMPLING:
        pair_counts = Counter(zip(ysent_tr.tolist(), ypol_tr.tolist()))
        counts = np.array(list(pair_counts.values()), dtype=np.float32)
        med = float(np.median(counts)) if len(counts) else 1.0

        def inv_mult(c):
            if c <= 0: return JOINT_OVERSAMPLING_MAX_MULT
            m = med / float(c)
            return float(np.clip(m, 1.0, JOINT_OVERSAMPLING_MAX_MULT))

        inv_by_pair = {k: inv_mult(v) for k, v in pair_counts.items()}

        sample_weights = []
        for ys, yp in zip(ysent_tr, ypol_tr):
            inv = inv_by_pair.get((int(ys), int(yp)), 1.0)
            w = (1.0 - JOINT_ALPHA) * 1.0 + JOINT_ALPHA * inv
            sample_weights.append(w)

        sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
        trainer.set_train_sampler(sampler)
    elif USE_OVERSAMPLING:
        sample_weights = []
        for ys, yp in zip(ysent_tr, ypol_tr):
            ws = float(sent_w_np[ys]); wp = float(pol_w_np[yp])
            w = 0.5 * (ws + wp)
            sample_weights.append(w)
        sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
        trainer.set_train_sampler(sampler)

    trainer.train()

    # Eval on test
    test_out = trainer.predict(test_ds)
    metrics = {f"test_{k}": float(v) for k, v in test_out.metrics.items()}

    # Save artifacts
    trainer.save_model(run_dir)
    tokenizer.save_pretrained(run_dir)
    with open(os.path.join(run_dir, "metrics_test.json"), "w") as f:
        json.dump(metrics, f, indent=2)

    # Confusion matrices
    sent_logits, pol_logits = test_out.predictions
    ysent_pred = np.argmax(sent_logits, axis=1)
    ypol_pred  = np.argmax(pol_logits,  axis=1)

    cm_sent = confusion_matrix(ysent_te, ysent_pred, labels=list(range(num_sent_classes)))
    cm_pol  = confusion_matrix(ypol_te,  ypol_pred,  labels=list(range(num_pol_classes)))

    np.save(os.path.join(run_dir, "cm_sent.npy"), cm_sent)
    np.save(os.path.join(run_dir, "cm_pol.npy"),  cm_pol)

    # Quick plots
    def plot_cm(cm, labels, title, path_png):
        fig, ax = plt.subplots(figsize=(4.5, 4))
        im = ax.imshow(cm, interpolation="nearest")
        ax.set_title(title)
        ax.set_xlabel("Predicted")
        ax.set_ylabel("True")
        ax.set_xticks(range(len(labels))); ax.set_xticklabels(labels, rotation=45, ha="right")
        ax.set_yticks(range(len(labels))); ax.set_yticklabels(labels)
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                ax.text(j, i, cm[i, j], ha="center", va="center")
        fig.colorbar(im, ax=ax, fraction=0.046)
        plt.tight_layout()
        plt.savefig(path_png, dpi=160)
        plt.close(fig)

    plot_cm(cm_sent, sent_le.classes_, "Sentiment Confusion", os.path.join(run_dir, "cm_sent.png"))
    plot_cm(cm_pol,  pol_le.classes_,  "Polarization Confusion", os.path.join(run_dir, "cm_pol.png"))

    # Text reports
    rep_sent = classification_report(ysent_te, ysent_pred, target_names=sent_le.classes_, digits=4, zero_division=0)
    rep_pol  = classification_report(ypol_te,  ypol_pred,  target_names=pol_le.classes_,  digits=4, zero_division=0)
    with open(os.path.join(run_dir, "report_sentiment.txt"), "w") as f:
        f.write(rep_sent)
    with open(os.path.join(run_dir, "report_polarization.txt"), "w") as f:
        f.write(rep_pol)

    summary_row = {
        "model_key": model_key,
        "base_name": base_name,
        **metrics
    }
    return summary_row, (ysent_pred, ypol_pred)


## SECTION 10


In [12]:
# SECTION 10

results = []
pred_cache = {}

for key in MODELS_TO_RUN:
    print(f"\n=== Running {key} -> {MODEL_CONFIGS[key]['name']} ===")
    row, preds = train_eval_one_model(
        key,
        X_train, X_val, X_test,
        ysent_train, ysent_val, ysent_test,
        ypol_train,  ypol_val,  ypol_test,
        sent_weights_np, pol_weights_np
    )
    results.append(row)
    pred_cache[key] = preds

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(OUT_DIR, "summary_results.csv"), index=False)
results_df



=== Running mbert -> bert-base-multilingual-cased ===


Epoch,Training Loss,Validation Loss,Sent Acc,Sent Prec,Sent Rec,Sent F1,Pol Acc,Pol Prec,Pol Rec,Pol F1,Macro F1 Avg
1,1.736300,No log,0.640984,0.590243,0.590519,0.588728,0.427049,0.502777,0.528924,0.358799,0.473764
2,1.364300,No log,0.609836,0.584277,0.651213,0.597906,0.612295,0.541505,0.646276,0.545275,0.571591
3,0.985100,No log,0.652459,0.621169,0.662301,0.636251,0.635246,0.549544,0.650939,0.559808,0.598029
4,0.845000,No log,0.666393,0.626698,0.673139,0.644157,0.703279,0.590658,0.661273,0.611009,0.627583



=== Running xlm_roberta -> xlm-roberta-base ===


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Sent Acc,Sent Prec,Sent Rec,Sent F1,Pol Acc,Pol Prec,Pol Rec,Pol F1,Macro F1 Avg
1,1.839900,No log,0.586066,0.549233,0.610039,0.557293,0.450820,0.508368,0.565895,0.396564,0.476929
2,1.570700,No log,0.660656,0.607270,0.617606,0.609936,0.661475,0.563141,0.640373,0.580434,0.595185
3,1.136700,No log,0.686885,0.643465,0.671329,0.654851,0.677049,0.588646,0.682438,0.609117,0.631984
4,1.115200,No log,0.686066,0.641877,0.683929,0.657243,0.703279,0.600176,0.675555,0.621831,0.639537


,model_key,base_name,test_test_sent_acc,test_test_sent_prec,test_test_sent_rec,test_test_sent_f1,test_test_pol_acc,test_test_pol_prec,test_test_pol_rec,test_test_pol_f1,test_test_macro_f1_avg,test_test_runtime,test_test_samples_per_second,test_test_steps_per_second
0,mbert,bert-base-multilingual-cased,0.669672,0.627791,0.675722,0.645411,0.695082,0.575257,0.625226,0.590348,0.617879,4.0710,299.682,18.914
1,xlm_roberta,xlm-roberta-base,0.684426,0.636367,0.685988,0.653238,0.696721,0.573681,0.624418,0.588658,0.620948,4.8096,253.657,16.010


## SECTION 11


In [15]:
# ===== Section 11 — Detailed Breakdown Reports (per-class + cross-slices) =====
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import os
import json

def per_class_breakdown(y_true, y_pred, class_names):
    rep = classification_report(
        y_true, y_pred,
        target_names=list(class_names),
        output_dict=True, zero_division=0
    )
    # Keep only the class rows in the given order
    rows = []
    for cname in class_names:
        if cname in rep:
            rows.append({
                "class": cname,
                "precision": rep[cname]["precision"],
                "recall":    rep[cname]["recall"],
                "f1":        rep[cname]["f1-score"],
                "support":   int(rep[cname]["support"]),
            })
        else:
            rows.append({"class": cname, "precision": 0.0, "recall": 0.0, "f1": 0.0, "support": 0})
    return pd.DataFrame(rows)

def cross_slice_breakdown(
    slice_true,  # array of ints for the slicing label (e.g., true sentiment indices)
    slice_names, # names of the slicing label classes (e.g., sentiment class names)
    task_true,   # array of ints for the task we evaluate (e.g., true polarity indices)
    task_pred,   # array of ints for the task predictions (e.g., predicted polarity indices)
    task_names,  # names of the task classes (e.g., polarity class names)
    slice_label  # string for the slice axis name, e.g., "sentiment" or "polarity"
):
    """
    For each class s in slice_true, evaluate the task predictions on the subset where slice_true == s.
    Returns one row per slice value, including macro-F1, accuracy, and per-class F1 for the task.
    """
    rows = []
    for idx, sname in enumerate(slice_names):
        mask = (slice_true == idx)
        n = int(mask.sum())
        if n == 0:
            # No samples for this slice in test set
            row = {"slice": sname, "support": 0, "accuracy": np.nan, "macro_f1": np.nan}
            for tname in task_names:
                row[f"f1_{tname}"] = np.nan
            rows.append(row)
            continue

        rep = classification_report(
            task_true[mask], task_pred[mask],
            target_names=list(task_names),
            output_dict=True, zero_division=0
        )
        row = {
            "slice": sname,
            "support": n,
            "accuracy": rep["accuracy"],
            "macro_f1": rep["macro avg"]["f1-score"],
        }
        for tname in task_names:
            row[f"f1_{tname}"] = rep[tname]["f1-score"]
        rows.append(row)

    df = pd.DataFrame(rows)
    # Sort slices by support (desc) for readability
    df = df.sort_values(by="support", ascending=False).reset_index(drop=True)
    return df

# Where to save things
DETAILS_DIR = os.path.join(OUT_DIR, "details")
os.makedirs(DETAILS_DIR, exist_ok=True)

all_breakdowns = {}

for key in MODELS_TO_RUN:
    print(f"\n=== Detailed breakdowns for {key} ===")
    ysent_pred, ypol_pred = pred_cache[key]

    # ---- Per-class reports on the full test set
    sent_per_class = per_class_breakdown(ysent_test, ysent_pred, sent_le.classes_)
    pol_per_class  = per_class_breakdown(ypol_test,  ypol_pred,  pol_le.classes_)

    # Save + show
    sent_csv = os.path.join(DETAILS_DIR, f"{key}_sentiment_per_class.csv")
    pol_csv  = os.path.join(DETAILS_DIR, f"{key}_polarization_per_class.csv")
    sent_per_class.to_csv(sent_csv, index=False)
    pol_per_class.to_csv(pol_csv, index=False)

    print("\nSentiment — per class (precision/recall/F1/support):")
    display(sent_per_class)

    print("\nPolarization — per class (precision/recall/F1/support):")
    display(pol_per_class)

    # ---- Cross-slice reports
    # Polarity performance within each (true) sentiment slice
    pol_given_sent = cross_slice_breakdown(
        slice_true=ysent_test, slice_names=sent_le.classes_,
        task_true=ypol_test,   task_pred=ypol_pred, task_names=pol_le.classes_,
        slice_label="sentiment"
    )
    pol_given_sent_csv = os.path.join(DETAILS_DIR, f"{key}_polarity_given_sentiment.csv")
    pol_given_sent.to_csv(pol_given_sent_csv, index=False)

    print("\nPolarity performance within each Sentiment slice (accuracy / macro-F1 / per-class F1):")
    display(pol_given_sent)

    # Sentiment performance within each (true) polarity slice
    sent_given_pol = cross_slice_breakdown(
        slice_true=ypol_test,  slice_names=pol_le.classes_,
        task_true=ysent_test,  task_pred=ysent_pred, task_names=sent_le.classes_,
        slice_label="polarity"
    )
    sent_given_pol_csv = os.path.join(DETAILS_DIR, f"{key}_sentiment_given_polarity.csv")
    sent_given_pol.to_csv(sent_given_pol_csv, index=False)

    print("\nSentiment performance within each Polarity slice (accuracy / macro-F1 / per-class F1):")
    display(sent_given_pol)

    # Keep for a single JSON bundle if you like
    all_breakdowns[key] = {
        "sentiment_per_class_csv": sent_csv,
        "polarization_per_class_csv": pol_csv,
        "polarity_given_sentiment_csv": pol_given_sent_csv,
        "sentiment_given_polarity_csv": sent_given_pol_csv
    }

# Optional: write an index JSON pointing to all CSVs
with open(os.path.join(DETAILS_DIR, "index.json"), "w") as f:
    json.dump(all_breakdowns, f, indent=2)
print("\nSaved detailed breakdowns to:", DETAILS_DIR)



=== Detailed breakdowns for mbert ===

Sentiment — per class (precision/recall/F1/support):


,class,precision,recall,f1,support
0,negative,0.789809,0.704545,0.744745,704
1,neutral,0.523944,0.542274,0.532951,343
2,positive,0.569620,0.780347,0.658537,173



Polarization — per class (precision/recall/F1/support):


,class,precision,recall,f1,support
0,non_polarized,0.538462,0.691860,0.605598,344
1,objective,0.319328,0.463415,0.378109,82
2,partisan,0.867982,0.720403,0.787337,794



Polarity performance within each Sentiment slice (accuracy / macro-F1 / per-class F1):


...


Polarity performance within each Sentiment slice (accuracy / macro-F1 / per-class F1):


,slice,support,accuracy,macro_f1,f1_non_polarized,f1_objective,f1_partisan
0,negative,704,0.762784,0.553914,0.525253,0.280702,0.855787
1,neutral,343,0.597668,0.550611,0.675824,0.422018,0.553991
2,positive,173,0.624277,0.547129,0.561983,0.358974,0.720430



Sentiment performance within each Polarity slice (accuracy / macro-F1 / per-class F1):


,slice,support,accuracy,macro_f1,f1_negative,f1_neutral,f1_positive
0,partisan,794,0.734257,0.630500,0.822669,0.391667,0.677165
1,non_polarized,344,0.587209,0.600414,0.550218,0.579268,0.671756
2,objective,82,0.609756,0.583821,0.473684,0.666667,0.611111



Saved detailed breakdowns to: ./runs_multitask/details


## SECTION 12

In [14]:
# ===== Section 12 — Length Diagnostics (clean) =====
import warnings

def token_lengths_summary(texts, titles, tokenizer, n=5000):
    # Random sample (or full if dataset is small)
    n = min(n, len(texts))
    idx = np.random.choice(len(texts), size=n, replace=False) if len(texts) > n else np.arange(len(texts))

    lengths = []
    # Silence the "sequence > 512" warnings emitted by some tokenizers for inspection
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="Token indices sequence length is longer.*")
        for i in idx:
            s = f"{titles[i]} [SEP] {texts[i]}"
            # We want raw length pre-truncation to choose MAX_LENGTH wisely
            ids = tokenizer.encode(s, add_special_tokens=True, truncation=False)
            lengths.append(len(ids))

    arr = np.array(lengths)
    stats = {
        "mean": float(arr.mean()),
        "p50":  float(np.percentile(arr, 50)),
        "p90":  float(np.percentile(arr, 90)),
        "p95":  float(np.percentile(arr, 95)),
        "p99":  float(np.percentile(arr, 99)),
        "max":  int(arr.max())
    }
    print("Token length stats:", stats)
    return stats

for key in MODELS_TO_RUN:
    name = MODEL_CONFIGS[key]["name"]
    tok = AutoTokenizer.from_pretrained(name)
    print(f"\n[{key}] {name}")
    token_lengths_summary(
        texts=X_train[TEXT_COL].values,
        titles=X_train[TITLE_COL].values,
        tokenizer=tok,
        n=5000
    )

# Tip:
# If p95 is comfortably < 192, you're fine. If you see p95 > 192, consider MAX_LENGTH=224
# (Update in Section 3 if you decide to bump it.)



[mbert] bert-base-multilingual-cased


Token indices sequence length is longer than the specified maximum sequence length for this model (916 > 512). Running this sequence through the model will result in indexing errors


Token length stats: {'mean': 111.2016, 'p50': 99.0, 'p90': 183.0, 'p95': 197.0, 'p99': 234.0, 'max': 916}

[xlm_roberta] xlm-roberta-base


Token indices sequence length is longer than the specified maximum sequence length for this model (950 > 512). Running this sequence through the model will result in indexing errors


Token length stats: {'mean': 108.6914, 'p50': 97.0, 'p90': 171.0, 'p95': 184.0, 'p99': 220.0, 'max': 950}
